# 1-node extended repressilator

This repressilator includes additional "step" - signalling molecule that acts as an intermediate between 2 repressors.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# AiiA expressed continuosly

#### code

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1)
genetic_n.add_regulator(aiia)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1, init_concentration=10)
tetr = lc.Regulator(name='TetR', degradation_rate=1)
ci = lc.Regulator(name='cI', degradation_rate=1)

# Signalling molecules
c4 = lc.Regulator(name='C4')

genetic_n.add_regulator([tetr, laci, ci, c4])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1)
genetic_n.add_regulator(rhli)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
laci_not_tetr = lc.Hill1(name='Plac', input=laci, output=[tetr, mcherry], alpha=[10000, 0], K=1, n=2)
tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[10000, 0], K=1, n=2)
# extended
ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=rhli, alpha=[10000, 0], K=1, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=0.001)
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, 10000], K=1, n=2)

# continuous expression of AiiA, one copy
j23100 = lc.Source(output=aiia, rate=1, name='J23100')

genetic_n.add_operator([c4_laci, laci_not_tetr, tetr_not_cini, ci_not_rhli, j23100])
genetic_n.add_impactor(rhli_c4)

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=c4, Km=10000, k2=2)
genetic_n.add_impactor(lactonase)


In [ ]:
plt.figure(figsize=(5,5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=10, mode="track_all")

#### results

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
fig5, ax5 = plt.subplots()
fig6, ax6 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
# m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax5)
# m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax5)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
# m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
# m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax4)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI', 'TetR', 'cI'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')
ax4.legend(['AiiA'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

# ax5.legend(['CFP', 'mVenus'])
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax5)
ax5.legend(['RFP'])
ax5.set_xlabel('Time (h)')
ax5.set_ylabel('Concentration (M)')

# ax6.legend(['LacI', 'cI'])
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
ax6.legend(['TetR'])
ax6.set_xlabel('Time (h)')
ax6.set_ylabel('Concentration (M)')

In [ ]:
# import loica as lc
# import matplotlib.pyplot as plt
# import networkx as nx
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# With diffusion

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1.02)
genetic_n.add_regulator(aiia)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1.02, init_concentration=10**-7)
tetr = lc.Regulator(name='TetR', degradation_rate=1.02)
ci = lc.Regulator(name='cI', degradation_rate=1.02)

# Signalling molecules
c4 = lc.Regulator(name='C4', diffusion_rate=180)

genetic_n.add_regulator([tetr, laci, ci, c4])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1.02)
genetic_n.add_regulator(rhli)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1.02, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1.02, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1.02, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
laci_not_tetr = lc.Hill1(name='Plac', input=laci, output=[tetr, mcherry], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
# extended
ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=rhli, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=960)
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)

# continuous expression of AiiA, one copy
j23100 = lc.Source(output=aiia, rate=3*2.56*10**-7, name='J23100')

genetic_n.add_operator([c4_laci, laci_not_tetr, tetr_not_cini, ci_not_rhli, j23100])
genetic_n.add_impactor(rhli_c4)

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=c4, Km=0.00511, k2=1.35*10**5)
genetic_n.add_impactor(lactonase)


In [ ]:
plt.figure(figsize=(5,5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=100, mode="track_all")

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
fig5, ax5 = plt.subplots()
fig6, ax6 = plt.subplots()
fig7, ax7 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax5)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax5)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='Extracellular C4'].plot(x='Time', y='Measurement', style='-', ax=ax7)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax4)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI', 'TetR', 'cI'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')
ax4.legend(['AiiA'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

ax5.legend(['CFP', 'mVenus'])
ax5.set_xlabel('Time (h)')
ax5.set_ylabel('Concentration (M)')

ax6.legend(['LacI', 'cI'])
ax6.set_xlabel('Time (h)')
ax6.set_ylabel('Concentration (M)')

ax7.legend(['C4'])
ax7.set_xlabel('Time (h)')
ax7.set_ylabel('Concentration (M)')

In [ ]:
# import loica as lc
# import matplotlib.pyplot as plt
# import networkx as nx
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# Continuous RhlI

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1.02)
genetic_n.add_regulator(aiia)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1.02, init_concentration=10**-7)
tetr = lc.Regulator(name='TetR', degradation_rate=1.02)
ci = lc.Regulator(name='cI', degradation_rate=1.02)

# Signalling molecules
c4 = lc.Regulator(name='C4')

genetic_n.add_regulator([tetr, laci, ci, c4])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1.02)
genetic_n.add_regulator(rhli)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1.02, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1.02, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1.02, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
laci_not_tetr = lc.Hill1(name='Plac', input=laci, output=[tetr, mcherry], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
# extended
ci_not_aiia = lc.Hill1(name='PcI', input=ci, output=aiia, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=960)
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)

# continuous expression of RhlI, one copy
j23117 = lc.Source(output=rhli, rate=1.54*10**-8, name='J23117')

genetic_n.add_operator([c4_laci, laci_not_tetr, tetr_not_cini, ci_not_aiia, j23117])
genetic_n.add_impactor(rhli_c4)

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=c4, Km=0.00511, k2=1.35*10**5)
genetic_n.add_impactor(lactonase)


In [ ]:
plt.figure(figsize=(5,5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=10, mode="track_all")

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
fig5, ax5 = plt.subplots()
fig6, ax6 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax5)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax5)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax4)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI', 'TetR', 'cI'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')
ax4.legend(['AiiA'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

ax5.legend(['CFP', 'mVenus'])
ax5.set_xlabel('Time (h)')
ax5.set_ylabel('Concentration (M)')

ax6.legend(['LacI', 'cI'])
ax6.set_xlabel('Time (h)')
ax6.set_ylabel('Concentration (M)')

In [ ]:
# import loica as lc
# import matplotlib.pyplot as plt
# import networkx as nx
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# Continuous RhlI with diffusion

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1.02)
genetic_n.add_regulator(aiia)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1.02, init_concentration=10**-7)
tetr = lc.Regulator(name='TetR', degradation_rate=1.02)
ci = lc.Regulator(name='cI', degradation_rate=1.02)

# Signalling molecules
c4 = lc.Regulator(name='C4', diffusion_rate=180)

genetic_n.add_regulator([tetr, laci, ci, c4])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1.02)
genetic_n.add_regulator(rhli)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1.02, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1.02, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1.02, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
laci_not_tetr = lc.Hill1(name='Plac', input=laci, output=[tetr, mcherry], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
# extended
ci_not_aiia = lc.Hill1(name='PcI', input=ci, output=aiia, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=960)
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)

# continuous expression of RhlI, one copy
j23117 = lc.Source(output=rhli, rate=1.54*10**-8, name='J23117')

genetic_n.add_operator([c4_laci, laci_not_tetr, tetr_not_cini, ci_not_aiia, j23117])
genetic_n.add_impactor(rhli_c4)

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=c4, Km=0.00511, k2=1.35*10**5)
genetic_n.add_impactor(lactonase)


In [ ]:
plt.figure(figsize=(5,5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=100, mode="track_all")

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
fig5, ax5 = plt.subplots()
fig6, ax6 = plt.subplots()
fig7, ax7 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax5)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax5)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='Extracellular C4'].plot(x='Time', y='Measurement', style='-', ax=ax7)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax4)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI', 'TetR', 'cI'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')
ax4.legend(['AiiA'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

ax5.legend(['CFP', 'mVenus'])
ax5.set_xlabel('Time (h)')
ax5.set_ylabel('Concentration (M)')

ax6.legend(['LacI', 'cI'])
ax6.set_xlabel('Time (h)')
ax6.set_ylabel('Concentration (M)')

ax7.legend(['C4'])
ax7.set_xlabel('Time (h)')
ax7.set_ylabel('Concentration (M)')